In [1]:
# 🛠️ Step 1: Project Setup and Mock Calendar Initialization

# ✅ Import required libraries
import pandas as pd
from datetime import datetime, timedelta
import uuid
import json

# ✅ Configuration for mock calendar data
# Let's simulate a dictionary where keys are dates and values are calendar events

def generate_mock_calendar(start_date: str, days: int = 7):
    """
    Generate mock calendar data for a week from a given start date.
    Each day has 1–2 events randomly placed.
    """
    import random

    calendar_data = {}
    base_date = datetime.strptime(start_date, "%Y-%m-%d")
    for i in range(days):
        current_date = (base_date + timedelta(days=i)).date().isoformat()
        events = []
        num_events = random.choice([1, 2])
        start_hour = 9
        for j in range(num_events):
            event_start = datetime(base_date.year, base_date.month, base_date.day + i, start_hour + j*2)
            event_end = event_start + timedelta(hours=1)
            events.append({
                "id": str(uuid.uuid4()),
                "title": f"Project Work {j+1}",
                "start": event_start.isoformat(),
                "end": event_end.isoformat()
            })
        calendar_data[current_date] = events
    return calendar_data

# ✅ Generate mock calendar for demo
mock_calendar = generate_mock_calendar("2025-08-01", days=7)

# ✅ Preview the mock calendar data
import pprint
pprint.pprint(mock_calendar)

{'2025-08-01': [{'end': '2025-08-01T10:00:00',
                 'id': 'd6e601b4-d1ea-4d5f-bfd9-f953cd4b45c8',
                 'start': '2025-08-01T09:00:00',
                 'title': 'Project Work 1'}],
 '2025-08-02': [{'end': '2025-08-02T10:00:00',
                 'id': 'a9d49b2c-2b90-465e-8b48-1deb3b40c809',
                 'start': '2025-08-02T09:00:00',
                 'title': 'Project Work 1'},
                {'end': '2025-08-02T12:00:00',
                 'id': '2a5492a0-4507-421e-a1bd-4b646deb8ba5',
                 'start': '2025-08-02T11:00:00',
                 'title': 'Project Work 2'}],
 '2025-08-03': [{'end': '2025-08-03T10:00:00',
                 'id': '836558ce-1272-4e42-ad44-4b1d5fa5d154',
                 'start': '2025-08-03T09:00:00',
                 'title': 'Project Work 1'}],
 '2025-08-04': [{'end': '2025-08-04T10:00:00',
                 'id': '96a75d39-86c5-4d4e-b98e-9898ec2aaa1d',
                 'start': '2025-08-04T09:00:00',
                 'titl

In [2]:
# 📄 Create a sample timesheet CSV for testing
sample_data = {
    'date': ['2025-08-01', '2025-08-01', '2025-08-02'],
    'start': ['09:00', '11:00', '10:00'],
    'end': ['10:00', '12:00', '11:00'],
    'project': ['Project Alpha', 'Project Beta', 'Project Gamma']
}

sample_df = pd.DataFrame(sample_data)
sample_df.to_csv('timesheet.csv', index=False)

print("✅ Sample timesheet.csv created.")

✅ Sample timesheet.csv created.


In [3]:
# 📥 Step 2: Load and Parse Timesheet CSV File

# ✅ Sample structure of expected CSV
# Columns: date (YYYY-MM-DD), start (HH:MM), end (HH:MM), project

# Example:
# date,start,end,project
# 2025-08-01,09:00,10:00,Project Alpha
# 2025-08-01,11:00,12:00,Project Beta

# ✅ Load CSV file
# (Replace 'timesheet.csv' with your actual CSV file path)
timesheet_df = pd.read_csv('timesheet.csv')

# ✅ Convert time strings to datetime objects
def parse_timesheet(df):
    df['date'] = pd.to_datetime(df['date']).dt.date
    df['start'] = pd.to_datetime(df['start'], format="%H:%M").dt.time
    df['end'] = pd.to_datetime(df['end'], format="%H:%M").dt.time
    return df

timesheet_df = parse_timesheet(timesheet_df)

# ✅ Preview the parsed timesheet
print("✅ Parsed Timesheet Data:")
display(timesheet_df)

✅ Parsed Timesheet Data:


,date,start,end,project
0,2025-08-01,09:00:00,10:00:00,Project Alpha
1,2025-08-01,11:00:00,12:00:00,Project Beta
2,2025-08-02,10:00:00,11:00:00,Project Gamma


In [4]:
# 📆 Step 3: Calendar Event Fetch Function (simulating GET /calendar/events?date=...)

def get_calendar_events_by_date(date: str, calendar_data: dict):
    """
    Simulates a GET API call to fetch calendar events by date.
    
    Args:
        date (str): Date in 'YYYY-MM-DD' format.
        calendar_data (dict): Dictionary of calendar events.
    
    Returns:
        list: List of event dicts for that date, or empty list if none.
    """
    try:
        parsed_date = datetime.strptime(date, "%Y-%m-%d").date().isoformat()
        events = calendar_data.get(parsed_date, [])
        return events
    except ValueError as ve:
        print(f"Invalid date format: {ve}")
        return []

# ✅ Test the function with a sample date
sample_date = '2025-08-01'
events = get_calendar_events_by_date(sample_date, mock_calendar)

# ✅ Pretty print the events
print(f"📅 Events on {sample_date}:")
pprint.pprint(events)

📅 Events on 2025-08-01:
[{'end': '2025-08-01T10:00:00',
  'id': 'd6e601b4-d1ea-4d5f-bfd9-f953cd4b45c8',
  'start': '2025-08-01T09:00:00',
  'title': 'Project Work 1'}]


In [5]:
# 🔍 Step 4: Validation Logic – Flag Missing & Extra Entries

from datetime import datetime, date, time

def time_overlap(start1, end1, start2, end2):
    """
    Check if two time intervals overlap.
    All parameters are datetime.time objects.
    """
    return max(start1, start2) < min(end1, end2)

def validate_timesheet_against_calendar(timesheet_df, calendar_data):
    """
    Validate timesheet entries against calendar events and flag discrepancies.
    
    Returns:
        dict: {
            "missingEntries": [...],
            "extraEntries": [...]
        }
    """
    missing_entries = []
    extra_entries = []

    # Group timesheet entries by date
    grouped_ts = timesheet_df.groupby('date')

    for day, events in calendar_data.items():
        cal_date = datetime.strptime(day, "%Y-%m-%d").date()

        ts_entries = grouped_ts.get_group(cal_date) if cal_date in grouped_ts.groups else pd.DataFrame(columns=timesheet_df.columns)
        matched_ts = [False] * len(ts_entries)
        matched_cal = [False] * len(events)

        # Compare each calendar event with timesheet entries
        for i, cal_event in enumerate(events):
            cal_start = datetime.fromisoformat(cal_event['start']).time()
            cal_end = datetime.fromisoformat(cal_event['end']).time()
            match_found = False

            for j, ts_row in ts_entries.iterrows():
                ts_start = ts_row['start']
                ts_end = ts_row['end']

                if time_overlap(cal_start, cal_end, ts_start, ts_end):
                    match_found = True
                    matched_ts[ts_entries.index.get_loc(j)] = True
                    matched_cal[i] = True
                    break

            if not match_found:
                missing_entries.append({
                    "date": cal_date.isoformat(),
                    "start": cal_start.isoformat(),
                    "end": cal_end.isoformat(),
                    "reason": "No matching timesheet entry"
                })

        # Identify extra timesheet entries
        for i, matched in enumerate(matched_ts):
            if not matched:
                row = ts_entries.iloc[i]
                extra_entries.append({
                    "date": row['date'].isoformat(),
                    "start": row['start'].isoformat(),
                    "end": row['end'].isoformat(),
                    "project": row['project'],
                    "reason": "No matching calendar event"
                })

    return {
        "missingEntries": missing_entries,
        "extraEntries": extra_entries
    }

# ✅ Run the validation
validation_report = validate_timesheet_against_calendar(timesheet_df, mock_calendar)

# ✅ Preview the results
print("🚨 Missing Entries from Timesheet:")
pprint.pprint(validation_report["missingEntries"])

print("\n⚠️ Extra Entries in Timesheet:")
pprint.pprint(validation_report["extraEntries"])

🚨 Missing Entries from Timesheet:
[{'date': '2025-08-02',
  'end': '10:00:00',
  'reason': 'No matching timesheet entry',
  'start': '09:00:00'},
 {'date': '2025-08-02',
  'end': '12:00:00',
  'reason': 'No matching timesheet entry',
  'start': '11:00:00'},
 {'date': '2025-08-03',
  'end': '10:00:00',
  'reason': 'No matching timesheet entry',
  'start': '09:00:00'},
 {'date': '2025-08-04',
  'end': '10:00:00',
  'reason': 'No matching timesheet entry',
  'start': '09:00:00'},
 {'date': '2025-08-05',
  'end': '10:00:00',
  'reason': 'No matching timesheet entry',
  'start': '09:00:00'},
 {'date': '2025-08-05',
  'end': '12:00:00',
  'reason': 'No matching timesheet entry',
  'start': '11:00:00'},
 {'date': '2025-08-06',
  'end': '10:00:00',
  'reason': 'No matching timesheet entry',
  'start': '09:00:00'},
 {'date': '2025-08-07',
  'end': '10:00:00',
  'reason': 'No matching timesheet entry',
  'start': '09:00:00'},
 {'date': '2025-08-07',
  'end': '12:00:00',
  'reason': 'No matching 

In [6]:
# 🧾 Step 5: Simulate GET /reports/{id}

# ✅ In-memory "report storage"
report_storage = {}

# ✅ Save report with a unique ID
def save_report(report_dict):
    report_id = str(uuid.uuid4())
    report_storage[report_id] = report_dict
    return report_id

# ✅ Fetch report by ID
def get_report_by_id(report_id):
    report = report_storage.get(report_id)
    if report:
        return json.dumps(report, indent=2, default=str)
    else:
        return json.dumps({"error": "Report not found"}, indent=2)

# ✅ Store the validation report
report_id = save_report(validation_report)

print(f"✅ Report saved with ID: {report_id}")

# ✅ Simulate fetching the report via GET /reports/{id}
fetched_report_json = get_report_by_id(report_id)
print("📦 Fetched Report JSON:")
print(fetched_report_json)

✅ Report saved with ID: 78e05428-3845-4c50-989d-83e4d0caa356
📦 Fetched Report JSON:
{
  "missingEntries": [
    {
      "date": "2025-08-02",
      "start": "09:00:00",
      "end": "10:00:00",
      "reason": "No matching timesheet entry"
    },
    {
      "date": "2025-08-02",
      "start": "11:00:00",
      "end": "12:00:00",
      "reason": "No matching timesheet entry"
    },
    {
      "date": "2025-08-03",
      "start": "09:00:00",
      "end": "10:00:00",
      "reason": "No matching timesheet entry"
    },
    {
      "date": "2025-08-04",
      "start": "09:00:00",
      "end": "10:00:00",
      "reason": "No matching timesheet entry"
    },
    {
      "date": "2025-08-05",
      "start": "09:00:00",
      "end": "10:00:00",
      "reason": "No matching timesheet entry"
    },
    {
      "date": "2025-08-05",
      "start": "11:00:00",
      "end": "12:00:00",
      "reason": "No matching timesheet entry"
    },
    {
      "date": "2025-08-06",
      "start": "09:00:00